# 用pym3包 （没安装上）

In [ ]:
import pandas as pd

pip install pymc3
pip install arviz

In [ ]:
import numpy as np
import pymc3 as pm
import arviz as az

In [3]:
# Given data
sku_costs = np.array([5, 6, 7, 8, 6, 2, 8, 5, 15, 7])
wtp_means = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
purchases = np.array([2, 16, 7, 3, 10, 4, 6, 19, 11, 12])
possible_std_devs = np.array([0.5, 1, 2, 3, 4])
n_customers = 100

# Define the model
with pm.Model() as model:
    # Priors
    std_dev_idx = pm.Categorical('std_dev_idx', p=np.ones(len(possible_std_devs))/len(possible_std_devs), shape=len(sku_costs))
    std_devs = pm.Deterministic('std_devs', possible_std_devs[std_dev_idx])

    # Likelihood
    for i in range(len(sku_costs)):
        pm.Normal('wtp_{}'.format(i), mu=wtp_means[i], sd=std_devs[i], observed=np.repeat(sku_costs[i], purchases[i]))
    
    # Posterior
    trace = pm.sample(5000, return_inferencedata=True)

# Summarize the results
az.summary(trace, var_names=['std_dev_idx'])
az.plot_posterior(trace, var_names=['std_dev_idx'], hdi_prob=0.95)

NameError: name 'pm' is not defined

# 只用numpy

In [7]:
import numpy as np
from scipy.stats import norm

# Given data
sku_costs = np.array([5, 6, 7, 8, 6, 2, 8, 5, 15, 7])
wtp_means = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
purchases = np.array([2, 16, 7, 3, 10, 4, 6, 19, 11, 12])
possible_std_devs = np.array([0.5, 1, 2, 3, 4])
prior_prob = 1 / len(possible_std_devs)  # Each standard deviation is equally likely

# Number of customers
n_customers = 100

# Function to compute likelihood
def compute_likelihood(sku_index, std_dev):
    mean = wtp_means[sku_index]
    cost = sku_costs[sku_index]
    purchase_count = purchases[sku_index]
    
    # Probability of purchase
    prob_purchase = norm.cdf(cost, loc=mean, scale=std_dev)
    
    # Likelihood of observing the purchase count
    likelihood = (prob_purchase ** purchase_count) * ((1 - prob_purchase) ** (n_customers - purchase_count))
    
    return likelihood

# Calculate the posterior probabilities
posterior_probs = np.zeros((len(sku_costs), len(possible_std_devs)))

for i in range(len(sku_costs)):
    likelihoods = np.array([compute_likelihood(i, std_dev) for std_dev in possible_std_devs])
    marginal_likelihood = np.sum(likelihoods * prior_prob)
    posterior_probs[i] = (likelihoods * prior_prob) / marginal_likelihood

# Print the posterior probabilities
for i in range(len(sku_costs)):
    print(f'SKU {i + 1} posterior probabilities for std deviations {possible_std_devs}:')
    print(posterior_probs[i])

SKU 1 posterior probabilities for std deviations [0.5 1.  2.  3.  4. ]:
[9.99979402e-01 2.05983321e-05 3.47866003e-13 5.93829611e-17
 4.35867028e-19]
SKU 2 posterior probabilities for std deviations [0.5 1.  2.  3.  4. ]:
[5.97178189e-15 9.40138734e-01 5.94067109e-02 4.38341034e-04
 1.62137305e-05]
SKU 3 posterior probabilities for std deviations [0.5 1.  2.  3.  4. ]:
[6.29882586e-08 9.99964599e-01 3.53284167e-05 9.80957146e-09
 5.56791812e-11]
SKU 4 posterior probabilities for std deviations [0.5 1.  2.  3.  4. ]:
[4.36343401e-04 9.99562379e-01 1.27804087e-06 5.44581780e-11
 9.37645756e-14]
SKU 5 posterior probabilities for std deviations [0.5 1.  2.  3.  4. ]:
[9.63004253e-08 9.99894911e-01 1.04884251e-04 1.06729163e-07
 1.49117667e-09]
SKU 6 posterior probabilities for std deviations [0.5 1.  2.  3.  4. ]:
[9.99999857e-01 1.43491900e-07 2.25720102e-12 3.36533719e-14
 3.71417857e-15]
SKU 7 posterior probabilities for std deviations [0.5 1.  2.  3.  4. ]:
[7.28178398e-10 9.99872561e-

In [2]:
import numpy as np
from scipy.stats import norm

# Given data
sku_costs = np.array([5, 6, 7, 8, 6, 2, 8, 5, 15, 7])
wtp_means = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
purchases = np.array([2, 16, 7, 3, 10, 4, 6, 19, 11, 12])
possible_std_devs = np.array([0.2, 0.5, 0.8, 1, 1.5, 2])
prior_prob = 1 / len(possible_std_devs)  # Each standard deviation is equally likely

# Number of customers
n_customers = 100

# Function to compute likelihood
def compute_likelihood(sku_index, std_dev):
    mean = wtp_means[sku_index]
    cost = sku_costs[sku_index]
    purchase_count = purchases[sku_index]
    
    # Probability of purchase
    prob_purchase = norm.cdf(cost, loc=mean, scale=std_dev)
    
    # Likelihood of observing the purchase count
    likelihood = (prob_purchase ** purchase_count) * ((1 - prob_purchase) ** (n_customers - purchase_count))
    
    return likelihood

# Calculate the posterior probabilities
posterior_probs = np.zeros((len(sku_costs), len(possible_std_devs)))

for i in range(len(sku_costs)):
    likelihoods = np.array([compute_likelihood(i, std_dev) for std_dev in possible_std_devs])
    marginal_likelihood = np.sum(likelihoods * prior_prob)
    posterior_probs[i] = (likelihoods * prior_prob) / marginal_likelihood

# Print the posterior probabilities
for i in range(len(sku_costs)):
    print(f'SKU {i + 1} posterior probabilities for std deviations {possible_std_devs}:')
    print(posterior_probs[i])

SKU 1 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5 2. ]:
[1.50866018e-09 9.96353882e-01 3.62559254e-03 2.05236509e-05
 4.81630813e-10 3.46604782e-13]
SKU 2 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5 2. ]:
[1.16197564e-125 3.00382705e-015 6.82016013e-003 4.72893052e-001
 4.90405006e-001 2.98817822e-002]
SKU 3 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5 2. ]:
[4.43622096e-73 4.42127792e-08 2.91498950e-01 7.01896115e-01
 6.58009336e-03 2.47977563e-05]
SKU 4 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5 2. ]:
[1.22847809e-40 1.54871013e-04 6.44602538e-01 3.54773871e-01
 4.68265596e-04 4.53614018e-07]
SKU 5 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5 2. ]:
[8.41157233e-77 6.28955975e-08 3.38202933e-01 6.53049949e-01
 8.67855278e-03 6.85018537e-05]
SKU 6 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5 2. ]:
[9.99991874e-01 8.12542821e-06 1.27802577e-10 1.16593330e-12
 9.44645200e

In [3]:
posterior_probs

array([[1.50866018e-009, 9.96353882e-001, 3.62559254e-003,
        2.05236509e-005, 4.81630813e-010, 3.46604782e-013],
       [1.16197564e-125, 3.00382705e-015, 6.82016013e-003,
        4.72893052e-001, 4.90405006e-001, 2.98817822e-002],
       [4.43622096e-073, 4.42127792e-008, 2.91498950e-001,
        7.01896115e-001, 6.58009336e-003, 2.47977563e-005],
       [1.22847809e-040, 1.54871013e-004, 6.44602538e-001,
        3.54773871e-001, 4.68265596e-004, 4.53614018e-007],
       [8.41157233e-077, 6.28955975e-008, 3.38202933e-001,
        6.53049949e-001, 8.67855278e-003, 6.85018537e-005],
       [9.99991874e-001, 8.12542821e-006, 1.27802577e-010,
        1.16593330e-012, 9.44645200e-016, 1.83407274e-017],
       [3.72230822e-082, 6.33680085e-010, 1.02352742e-001,
        8.70115526e-001, 2.74208497e-002, 1.10881714e-004],
       [5.77398812e-104, 1.10892833e-011, 3.95707165e-002,
        6.35742737e-001, 3.00147190e-001, 2.45393568e-002],
       [0.00000000e+000, 3.42606373e-084, 1.0181

In [5]:
import csv
with open('posterior_probs.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # 遍历数组的每一行
    for row in posterior_probs:
        writer.writerow(row)


# 修改likelihood 加上value

In [26]:
import numpy as np
from scipy.stats import norm, multinomial

# Given data
sku_costs = np.array([5, 6, 7, 8, 6, 2, 8, 5, 15, 7])
wtp_means = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
purchases = np.array([2, 16, 7, 3, 10, 4, 6, 19, 11, 12])
possible_std_devs = np.array([0.2, 0.5, 0.8, 1.0, 1.5])
prior_prob = 1 / len(possible_std_devs)  # Each standard deviation is equally likely

# Number of customers
n_customers = 100

# Function to compute the value distributions for each SKU
def compute_value_distributions(std_dev):
    values = np.zeros((n_customers, len(sku_costs)))
    for i in range(len(sku_costs)):
        mean = wtp_means[i]
        cost = sku_costs[i]
        values[:, i] = np.random.normal(loc=mean, scale=std_dev, size=n_customers) - cost
    return values

# Calculate the posterior probabilities
posterior_probs = np.zeros(len(possible_std_devs))

for std_dev in possible_std_devs:
    # Simulate value distributions
    values = compute_value_distributions(std_dev)
    
    # Determine purchases based on highest value
    chosen_skus = np.argmax(values, axis=1)
    
    # Count purchases for each SKU
    simulated_purchases = np.array([np.sum(chosen_skus == i) for i in range(len(sku_costs))])
    
    # Compute the likelihood using multinomial distribution
    likelihood = multinomial.pmf(purchases, n_customers, simulated_purchases / n_customers)
    posterior_probs[np.where(possible_std_devs == std_dev)[0][0]] = likelihood * prior_prob

# Normalize posterior probabilities
posterior_probs /= np.sum(posterior_probs)


# Print the posterior probabilities
for i in range(len(sku_costs)):
    print(f'SKU {i + 1} posterior probabilities for std deviations {possible_std_devs}:')
    print(posterior_probs[i])

# 修改value的定义

In [2]:
import numpy as np
from scipy.stats import norm, multinomial

# Given data
sku_costs = np.array([5, 6, 7, 8, 6, 2, 8, 5, 15, 7])
wtp_means = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
purchases = np.array([2, 16, 7, 3, 10, 4, 6, 19, 11, 12])
possible_std_devs = np.array([0.2, 0.5, 0.8, 1.0, 1.5])
prior_prob = 1 / len(possible_std_devs) 

# Number of customers
n_customers = 100

def compute_value_distribution_for_sku(sku_index, std_dev):
    mean = wtp_means[sku_index]
    cost = sku_costs[sku_index]
    values = np.random.normal(loc=mean, scale=std_dev, size=n_customers) - cost
    return values


compute_value_distribution_for_sku(2,2)

array([ 0.38410074,  1.70234837,  2.68439468, -1.5400381 ,  2.52735446,
        2.17907195,  5.32500532,  0.87184591,  2.61871114,  2.98893728,
       -1.49591934, -3.01613455, -1.8982282 ,  1.95926087,  1.13633625,
       -1.3049979 ,  2.36641933,  0.12760218,  4.24319443,  1.59225153,
        0.67280701,  1.7406106 ,  0.47208346, -2.27000493,  1.10735246,
       -1.29725284, -0.29354872,  1.90717908,  2.91256866,  5.91003855,
        0.01563035, -2.4259523 ,  3.7400363 ,  2.22053615,  3.60589823,
        3.97455969, -1.4036015 ,  1.29904422,  1.175483  ,  1.94945698,
        2.44970134,  2.7222264 ,  0.50060515, -2.38948952, -1.23808979,
        4.94876247,  2.22776273,  3.2114538 ,  1.49826768,  1.11698592,
        2.51358381, -1.15380742,  0.45876858,  0.69878144,  2.87782133,
        1.6779635 ,  1.86100306, -0.02729564,  0.83384496,  1.98886972,
       -0.33455776,  1.88917774,  3.79700104,  2.79587995,  1.75245683,
        3.06237317,  0.20581099, -1.693547  ,  3.11150992,  4.56

In [30]:
import numpy as np
from scipy.stats import norm, multinomial

# Given data
sku_costs = np.array([5, 6, 7, 8, 6, 2, 8, 5, 15, 7])
wtp_means = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
purchases = np.array([2, 16, 7, 3, 10, 4, 6, 19, 11, 12])
possible_std_devs = np.array([0.2, 0.5, 0.8, 1.0, 1.5])
prior_prob = 1 / len(possible_std_devs)  # Each standard deviation is equally likely

# Number of customers
n_customers = 100

# Function to compute the value distribution for a specific SKU
def compute_value_distribution_for_sku(sku_index, std_dev):
    mean = wtp_means[sku_index]
    cost = sku_costs[sku_index]
    values = np.random.normal(loc=mean, scale=std_dev, size=n_customers) - cost
    return values

# Function to compute the posterior probabilities
def compute_posterior_probabilities():
    posterior_probs = np.zeros(len(possible_std_devs))

    for std_dev in possible_std_devs:
        # Simulate value distributions for all SKUs
        values = np.zeros((n_customers, len(sku_costs)))
        for i in range(len(sku_costs)):
            values[:, i] = compute_value_distribution_for_sku(i, std_dev)
        
        # Determine purchases based on highest value
        chosen_skus = np.argmax(values, axis=1)
        
        # Count purchases for each SKU
        simulated_purchases = np.array([np.sum(chosen_skus == i) for i in range(len(sku_costs))])
        
        # Ensure probabilities sum to 1 for multinomial distribution
        simulated_purchases_prob = simulated_purchases / n_customers
        if np.sum(simulated_purchases_prob) != 1:
            simulated_purchases_prob = simulated_purchases_prob / np.sum(simulated_purchases_prob)
        
        # Compute the log-likelihood using multinomial distribution
        log_likelihood = multinomial.logpmf(purchases, n_customers, simulated_purchases_prob)
        posterior_probs[np.where(possible_std_devs == std_dev)[0][0]] = np.exp(log_likelihood) * prior_prob

    # Handle potential zero division
    if np.sum(posterior_probs) == 0:
        posterior_probs += 1e-10

    # Normalize posterior probabilities
    posterior_probs /= np.sum(posterior_probs)

    return posterior_probs

posterior_probs = compute_posterior_probabilities()

# Print the posterior probabilities
for i in range(len(sku_costs)):
    print(f'SKU {i + 1} posterior probabilities for std deviations {possible_std_devs}:')
    print(posterior_probs)

SKU 1 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5]:
[0.2 0.2 0.2 0.2 0.2]
SKU 2 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5]:
[0.2 0.2 0.2 0.2 0.2]
SKU 3 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5]:
[0.2 0.2 0.2 0.2 0.2]
SKU 4 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5]:
[0.2 0.2 0.2 0.2 0.2]
SKU 5 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5]:
[0.2 0.2 0.2 0.2 0.2]
SKU 6 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5]:
[0.2 0.2 0.2 0.2 0.2]
SKU 7 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5]:
[0.2 0.2 0.2 0.2 0.2]
SKU 8 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5]:
[0.2 0.2 0.2 0.2 0.2]
SKU 9 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5]:
[0.2 0.2 0.2 0.2 0.2]
SKU 10 posterior probabilities for std deviations [0.2 0.5 0.8 1.  1.5]:
[0.2 0.2 0.2 0.2 0.2]


In [37]:
compute_value_distribution_for_sku(2,0.5)

array([ 1.73092316,  1.01380521,  1.61573353,  0.57164464,  1.9289302 ,
        1.69188893,  1.02100118,  0.89132227,  1.28945834,  1.77010061,
        0.8365117 ,  1.7273777 ,  0.80298221,  1.02905957,  1.35359803,
        1.52200391,  1.01496679,  1.51951809,  1.73289625,  1.65670547,
        1.16015349,  2.02992748,  1.00517377,  2.51602103,  0.61892137,
        1.71100529,  1.87743138,  1.69375198,  0.84753024,  1.11514073,
        1.235081  ,  1.354096  ,  1.0563788 ,  1.68829896,  1.13631716,
        1.02708903,  1.8098185 ,  1.87669125,  2.33390337,  0.43981785,
        0.61766651,  1.74187266,  1.91222127,  0.62137895,  1.1458234 ,
        1.30749135,  0.50341666,  1.96965701,  2.5111736 ,  0.91080509,
        1.93819566,  1.33488739,  1.29649026,  1.13950927,  2.20280032,
        1.89469105,  1.72265827,  2.32405612,  2.17768798,  2.31215052,
        1.41692571,  1.9360128 ,  1.92769217,  1.06296045,  1.65664077,
        1.54775626,  1.88604291,  1.890036  ,  1.42874061,  0.80

# use combination

In [1]:
import numpy as np
from scipy.stats import multinomial
from itertools import product

# Given data
sku_costs = np.array([5, 6, 7, 8, 6, 2, 8, 5, 15, 7])
wtp_means = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
initial_price = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
purchases = np.array([2, 16, 7, 3, 10, 4, 6, 19, 11, 12])
possible_std_devs = np.array([0.2, 0.5, 0.8, 1, 1.5])
prior_prob = 1 / len(possible_std_devs)  # Each standard deviation is equally likely

# Number of customers
n_customers = 100

# Function to compute the value distribution for a specific SKU
def compute_value_distribution_for_sku(sku_index, std_dev):
    mean = wtp_means[sku_index]
    cost = sku_costs[sku_index]
    values = np.random.normal(loc=mean, scale=std_dev, size=n_customers) - initial_price
    return values

# Function to compute the posterior probabilities
def compute_posterior_probabilities():
    # Generate all possible standard deviation combinations for all SKUs
    std_dev_combinations = list(product(possible_std_devs, repeat=len(sku_costs)))
    
    posterior_probs = np.zeros((len(sku_costs), len(possible_std_devs)))

    for idx, std_dev_comb in enumerate(std_dev_combinations):
        # Simulate value distributions for all SKUs based on the current std_dev_comb
        values = np.zeros((n_customers, len(sku_costs)))
        for i in range(len(sku_costs)):
            values[:, i] = compute_value_distribution_for_sku(i, std_dev_comb[i])
        
        # Determine purchases based on highest value
        chosen_skus = np.argmax(values, axis=1)
        
        # Count purchases for each SKU
        simulated_purchases = np.array([np.sum(chosen_skus == i) for i in range(len(sku_costs))])
        
        # Ensure probabilities sum to 1 for multinomial distribution
        simulated_purchases_prob = simulated_purchases / n_customers
        if np.sum(simulated_purchases_prob) != 1:
            simulated_purchases_prob = simulated_purchases_prob / np.sum(simulated_purchases_prob)
        
        # Compute the log-likelihood using multinomial distribution
            log_likelihood = multinomial.logpmf([simulated_purchases, n_customers - simulated_purchases], n_customers, [simulated_purchases_prob, 1 - simulated_purchases_prob])
            posterior_probs[idx, std_dev_comb] = np.exp(log_likelihood) * prior_prob

        # Normalize posterior probabilities for the current SKU
        posterior_probs[idx, :] /= np.sum(posterior_probs[idx, :])

        return posterior_probs


compute_posterior_probabilities()

# Print the posterior probabilities
for i in range(len(sku_costs)):
    print(f'SKU {i + 1} posterior probabilities for std deviations {possible_std_devs}:')
    print(posterior_probs)

# 增加顾客不购买的情况，更改likelihood

In [ ]:
import numpy as np
from scipy.stats import multinomial
from itertools import product

# Given data
sku_costs = np.array([5, 6, 7, 8, 6, 2, 8, 5, 15, 7])
wtp_means = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
initial_price = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
purchases = np.array([2, 16, 7, 3, 10, 4, 6, 19, 11, 12])
possible_std_devs = np.array([0.2, 0.5, 0.8, 1, 1.5])
prior_prob = 1 / len(possible_std_devs)  # Each standard deviation is equally likely

# Number of customers
n_customers = 100
purchases = np.append(purchases, n_customers - purchases.sum())

# Function to compute the value distribution for a specific SKU
def compute_value_distribution_for_sku(sku_index, std):
    mean = wtp_means[sku_index]
    values = np.random.normal(loc=mean, scale=std, size=n_customers) - initial_price[sku_index]
    return values

# Generate all possible standard deviation combinations for all SKUs
std_dev_combinations = list(product(possible_std_devs, repeat=len(sku_costs)))

posterior_probs = {sku_idx: np.zeros(len(possible_std_devs)) for sku_idx in range(len(sku_costs)+1)}


for idx, std_dev_comb in enumerate(std_dev_combinations):
    # Simulate value distributions for all SKUs based on the current std_dev_comb
    values = np.zeros((n_customers, len(sku_costs)))
    for i in range(len(sku_costs)):
        values[:, i] = compute_value_distribution_for_sku(i, std_dev_comb[i])
            
    # Determine purchases based on highest value
    max_values = np.max(values, axis=1)

    # Count purchases for each SKU
    chosen_skus = np.full(values.shape[0], -1, dtype=int)
    simulated_purchases = np.zeros(len(sku_costs) + 1)
    # 只有当最大 value 大于 0 时，顾客才会购买
    for i in range(values.shape[0]):
        if max_values[i] > 0:
            sku_idx = np.argmax(values[i, :])
            chosen_skus[i] = sku_idx
            simulated_purchases[sku_idx] += 1
        else:
            simulated_purchases[-1] += 1
                    
    # Ensure probabilities sum to 1 for multinomial distribution
    simulated_purchases_prob = simulated_purchases / n_customers
            
    # Compute the likelihood using multinomial distribution
    # log_likelihood = multinomial.logpmf(purchases, n_customers, simulated_purchases_prob)
    # posterior = np.exp(log_likelihood) * prior_prob

    # marginal
    for sku_idx in range(len(sku_costs)):
        other_std_devs = [std_dev_comb[i] for i in range(len(sku_costs)) if i != sku_idx]
        marginal_likelihood = np.sum([multinomial.logpmf(purchases, n_customers, simulated_purchases_prob) for _ in other_std_devs])
        posterior_probs[sku_idx][idx] = np.exp(marginal_likelihood) * prior_prob

    # Normalize posterior probabilities for the current SKU
    for sku_idx in range(len(sku_costs)):
        posterior_probs[sku_idx] /= posterior_probs[sku_idx].sum()


# Print the posterior probabilities
for sku_idx in range(len(sku_costs)):
    print(f"SKU {sku_idx} 的后验概率:", posterior_probs[sku_idx])

# Marginal

In [58]:
import numpy as np
from scipy.stats import multinomial
from itertools import product

# Given data
sku_costs = np.array([5, 6, 7, 8, 6, 2, 8, 5, 15, 7])
wtp_means = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
initial_price = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
purchases = np.array([2, 16, 7, 3, 10, 4, 6, 19, 11, 12])
possible_std_devs = np.array([0.2, 0.5, 0.8, 1, 1.5])
prior_prob = 1 / len(possible_std_devs)  # Each standard deviation is equally likely

# Number of customers
n_customers = 100
purchases = np.append(purchases, n_customers - purchases.sum())


In [29]:
# Function to compute the value distribution for a specific SKU
def compute_value_distribution_for_sku(sku_index, std):
    mean = wtp_means[sku_index]
    values = np.random.normal(loc=mean, scale=std, size=n_customers) - initial_price[sku_index]
    return values

In [ ]:
# Generate all possible standard deviation combinations for all SKUs
std_dev_combinations = list(product(possible_std_devs, repeat=len(sku_costs)))

# Initialize the posterior array
posterior = np.zeros((len(std_dev_combinations)),len(sku_costs)+1)

for idx, std_dev_comb in enumerate(std_dev_combinations):
        
    # Simulate value distributions for all SKUs based on the current std_dev_comb
    values = np.zeros((n_customers, len(sku_costs)))
    for i in range(len(sku_costs)):
        values[:, i] = compute_value_distribution_for_sku(i, std_dev_comb[i])
                
    # Determine purchases based on highest value
    max_values = np.max(values, axis=1)

    # Count purchases for each SKU
    chosen_skus = np.full(values.shape[0], -1, dtype=int)
    simulated_purchases = np.zeros(len(sku_costs) + 1)
    # 只有当最大 value 大于 0 时，顾客才会购买
    for i in range(values.shape[0]):
        if max_values[i] > 0:
            sku_idx = np.argmax(values[i, :])
            chosen_skus[i] = sku_idx
            simulated_purchases[sku_idx] += 1
        else:
            simulated_purchases[-1] += 1
                        
    # Ensure probabilities sum to 1 for multinomial distribution
    simulated_purchases_prob = simulated_purchases / n_customers
                
    # Compute the likelihood using multinomial distribution
    log_likelihood = multinomial.logpmf(purchases, n_customers, simulated_purchases_prob)
    unnormalized_posterior = np.exp(log_likelihood) * prior_prob
    posterior[idx, :] = unnormalized_posterior / unnormalized_posterior.sum()



In [ ]:

# Initialize the marginal posterior array
marginal_posterior = np.zeros((len(sku_costs)+1, len(possible_std_devs)))

# Calculate the marginal posterior probabilities for each SKU
for sku_idx in range(len(sku_costs)+1):
    for std_dev_idx, std_dev in enumerate(possible_std_devs):
        # Sum over all combinations where the current SKU has the current std_dev
        marginal_posterior[sku_idx, std_dev_idx] = np.sum(posterior[:, sku_idx][std_dev_combinations[:, sku_idx] == std_dev])

# Normalize the marginal posterior probabilities for each SKU
marginal_posterior /= np.sum(marginal_posterior, axis=1, keepdims=True)

In [44]:
from scipy.stats import multinomial

# 定义参数
x = [2, 3, 5]  # 观测到的事件次数
n = 10         # 试验总次数
p = [0.2, 0.3, 0.5]  # 每个类别的概率

# 计算对数概率质量函数
log_pmf = multinomial.logpmf(x, n, p)

likelihood = np.exp(log_pmf)
likelihood


np.float64(0.08504999999999999)

In [41]:
prior_prob = np.array([0.1, 0.4, 0.5])
unnormalized_posterior = likelihood * prior_prob

In [43]:
unnormalized_posterior / unnormalized_posterior.sum()

array([0.1, 0.4, 0.5])

# margin likelihood

In [2]:
import numpy as np
from scipy.stats import multinomial
from itertools import product
np.random.seed(0)
# Given data
sku_costs = np.array([5, 6, 7, 8, 6, 2, 8, 5, 15, 7])
wtp_means = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
initial_price = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
purchases = np.array([2, 16, 7, 3, 10, 4, 6, 19, 11, 12])
possible_std_devs = np.array([0.5, 1, 2])
prior_prob = 1 / len(possible_std_devs)  # Each standard deviation is equally likely

# Number of customers
n_customers = 100
purchases = np.append(purchases, n_customers - purchases.sum())


In [3]:
# Function to compute the value distribution for a specific SKU
def compute_value_distribution_for_sku(sku_index, std):
    mean = wtp_means[sku_index]
    values = np.random.normal(loc=mean, scale=std, size=n_customers) - initial_price[sku_index]
    return values

In [4]:
# Generate all possible standard deviation combinations for all SKUs
std_dev_combinations = list(product(possible_std_devs, repeat=len(sku_costs)))

# Initialize the posterior array
likelihood = np.zeros((len(std_dev_combinations)))

for idx, std_dev_comb in enumerate(std_dev_combinations):
        
    # Simulate value distributions for all SKUs based on the current std_dev_comb
    values = np.zeros((n_customers, len(sku_costs)))
    for i in range(len(sku_costs)):
        values[:, i] = compute_value_distribution_for_sku(i, std_dev_comb[i])
                
    # Determine purchases based on highest value
    max_values = np.max(values, axis=1)

    # Count purchases for each SKU
    chosen_skus = np.full(values.shape[0], -1, dtype=int)
    simulated_purchases = np.zeros(len(sku_costs) + 1)
    # 只有当最大 value 大于 0 时，顾客才会购买
    for i in range(values.shape[0]):
        if max_values[i] > 0:
            sku_idx = np.argmax(values[i, :])
            chosen_skus[i] = sku_idx
            simulated_purchases[sku_idx] += 1
        else:
            simulated_purchases[-1] += 1
                        
    # Ensure probabilities sum to 1 for multinomial distribution
    simulated_purchases_prob = simulated_purchases / n_customers

    # Ensure probabilities sum to 1 for multinomial distribution
    # epsilon = 1e-10
    # simulated_purchases_prob = (simulated_purchases + epsilon) / (n_customers + epsilon * len(sku_costs))

                
    # Compute the likelihood using multinomial distribution
    log_likelihood = multinomial.logpmf(purchases, n_customers, simulated_purchases_prob)
    print(log_likelihood, end='\t')
    likelihood[idx]= np.exp(log_likelihood)
    if idx == 100: break
    # posterior[idx, :] = unnormalized_posterior / unnormalized_posterior.sum()

-inf	-inf	-inf	-inf	-inf	-inf	-inf	-49.7612187950906	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-56.642110584354214	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-62.5768894425978	-77.88807258166275	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-78.41312704008061	-67.86613626694657	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	-inf	

In [83]:
np.isnan(likelihood).sum()

0

In [84]:
prior = np.full(len(possible_std_devs), 1 / len(possible_std_devs))

# Initialize the marginal likelihood array
marginal_likelihood = np.zeros((len(sku_costs)+1, len(possible_std_devs)))
marginal_likelihood_sum = np.zeros((len(sku_costs)+1))

# Calculate the marginal likelihood for each SKU and each assumption
posterior = pd.DataFrame(np.zeros((len(sku_costs)+1, len(possible_std_devs))))
for i in range(len(sku_costs) + 1):
    for j in range(len(possible_std_devs)):
        indices = [idx for idx, comb in enumerate(std_dev_combinations) if comb[i % len(comb)] == possible_std_devs[j]]
        marginal_likelihood[i, j] = np.sum(likelihood[indices])
    marginal_likelihood_sum[i] = np.sum(marginal_likelihood[i, :] * prior)
    posterior.iloc[i,:] = (marginal_likelihood[i,:] * prior_prob) / marginal_likelihood_sum[i]

In [85]:
posterior

,0,1,2
0,0.708938,0.287579,0.003482
1,0.000038,0.076618,0.923344
2,0.221079,0.703894,0.075027
3,0.745596,0.095790,0.158615
4,0.002355,0.799730,0.197915
5,0.279098,0.714269,0.006633
6,0.232710,0.763015,0.004275
7,0.000003,0.000870,0.999127
8,0.000400,0.789072,0.210528
9,0.074304,0.242442,0.683254


In [77]:
likelihood[-1]

np.float64(0.0)